In [33]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np


from tensorflow.keras.layers import LSTM

In [19]:
df_spectra=pd.read_csv("spectrum_energy_intensity.csv",index_col=[0,1])
df_input=pd.read_csv("input_data_table.csv",index_col=0)
df_input.head()


,k6a1,k6a2,k11,k12,k9a1,k9a2
0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
1,-0.5,-0.5,-0.5,-0.5,-0.5,0.0
2,-0.5,-0.5,-0.5,-0.5,-0.5,0.5
3,-0.5,-0.5,-0.5,-0.5,0.0,-0.5
4,-0.5,-0.5,-0.5,-0.5,0.0,0.0


In [20]:
test3=df_spectra.groupby(level=1).size()[0] #this gives the number of spectra in the dataset
print(test3)

#this works to get a matrix for each spectrum 
test2=df_spectra[df_spectra.index.isin(["spectrum0"],level=0)]["Energy"].to_numpy()
print(test2)

test4=df_input[df_input.index.isin([0],level=0)].to_numpy()[0]
print(test4)

470
[-0.758 -0.69  -0.612 -0.472 -0.406 -0.328 -0.152 -0.068 -0.014  0.134
  0.222  0.294  0.472  0.524  0.602  0.748  0.824  0.882]
[-0.5 -0.5 -0.5 -0.5 -0.5 -0.5]


In [21]:
list_spectra_Energy_array=[]
list_spectra_Intensity_array=[]
list_input=[]
#list_k6a1=df_input["k6a1"].to_numpy()
#list_k6a2=df_input["k6a2"].to_numpy()
#list_k11=df_input["k11"].to_numpy()
#list_k12=df_input["k12"].to_numpy()
#list_k9a1=df_input["k9a1"].to_numpy()
#list_k9a2=df_input["k9a2"].to_numpy()
for i in range(df_spectra.groupby(level=1).size()[0]):
    #list_spectra_Energy_array.append(df_spectra[df_spectra.index.isin(["spectrum"
    #                                                                   +str(i)],level=0)]["Energy"].to_numpy())
    #list_spectra_Intensity_array.append(df_spectra[df_spectra.index.isin(["spectrum"
    #                                                                      +str(i)],level=0)]["Intensity"].to_numpy())
    #list_input.append(df_input[df_input.index.isin([i],level=0)].to_numpy()[0])
    list_spectra_Energy_array.append([1,1,1])
    list_spectra_Intensity_array.append([1,1,1])
    list_input.append([1,1,1])

    
    
    
#list_spectra_Energy_array=np.asarray(list_spectra_Energy_array)   
#list_spectra_Intensity_array=np.asarray(list_spectra_Intensity_array)   
#list_input=np.asarray(list_input)   

#list_spectra_Energy_array=[x.tolist() for x in  list_spectra_Energy_array]
#list_spectra_Intensity_array=[x.tolist() for x in  list_spectra_Intensity_array]
#list_input=[x.tolist() for x in  list_input]

In [22]:
df_all=pd.DataFrame({'input_data':list_input,'Intensity':list_spectra_Intensity_array,'Energy':list_spectra_Energy_array})

In [23]:

df_training= df_all.sample(frac=0.8,random_state=10)
df_testing=df_all.drop(df_training.index)

list_spectra_Energy_array
list_spectra_Intensity_array
list_input
#print(list_spectra_Energy_array)
# array 2d 
#beispiel von phil
xbla=np.zeros([300,7])
for i in range(300):
    for j in range(6):
        if(j<3):
            xbla[i,j]=list_spectra_Energy_array[i][j]
        else:
            xbla[i,j]=list_spectra_Intensity_array[i][j-3]
            
ybla=np.zeros([300,3])            
for i in range(300):
    ybla[i]=list_input[i]  
           

In [24]:
df_testing.head(50)

,input_data,Intensity,Energy
4,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
8,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
11,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
13,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
15,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
16,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
18,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
31,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
33,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"
40,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]"


In [25]:
#create feature column

#I have no idea what i am doing 
feature_columns_A=[]
input_data = tf.feature_column.numeric_column("input")
feature_columns_A.append(input_data)

intensity = tf.feature_column.numeric_column("Intensity")
feature_columns_A.append(intensity)

energy = tf.feature_column.numeric_column("Energy")
feature_columns_A.append(energy)

my_feature_layer_A = tf.keras.layers.DenseFeatures(feature_columns_A)
#this does something

In [34]:
def create_model(my_learning_rate, my_feature_layer=0,unit_layer_1=10,unit_layer_2=12,regulations=0.04):
    """Create and compile a regression model."""
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    #model.add(my_feature_layer)

    # Describe the topography of the model by calling the tf.keras.layers.Dense
    # method once for each layer. We've specified the following arguments:
    #   * units specifies the number of nodes in this layer.
    #   * activation specifies the activation function (Rectified Linear Unit).
    #   * name is just a string that can be useful when debugging.
    model.add(LSTM(10))
    # Define the first hidden layer with 10 nodes.   
    model.add(tf.keras.layers.Dense(units=unit_layer_1, 
                                  activation='relu', 
                                  kernel_regularizer=tf.keras.regularizers.l1(regulations),
                                  name='Hidden1'))
  
    # Define the second hidden layer with 12 nodes. 
    model.add(tf.keras.layers.Dense(units=unit_layer_2, 
                                  activation='relu', 
                                  kernel_regularizer=tf.keras.regularizers.l1(regulations),
                                  name='Hidden2'))

    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=3,  
                                    name='Output'))                              
  
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])

    return model



In [35]:
def train_model(model, dataset, epochs, label_name,
                batch_size=None):
    """Train the model by feeding it data."""

    # Split the dataset into features and label.
    features = {name:np.array(value) for name, value in dataset.items()}
    print(type(features))
    label = np.array(features.pop(label_name))
    #history = model.fit(x=features, y=label, batch_size=batch_size,
    #                  epochs=epochs, shuffle=True) 
    history = model.fit(x=xbla, y=ybla, batch_size=batch_size,
                      epochs=epochs, shuffle=True) 

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch
  
    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch. 
    hist = pd.DataFrame(history.history)
    mse = hist["mean_squared_error"]

    return epochs, mse

In [36]:
#start the model
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 20
batch_size = 80


# Specify the label
label_name = "Energy"

# Establish the model's topography.
my_model = create_model(learning_rate)
epochs, mse = train_model(my_model, df_training, epochs, 
                          label_name,batch_size)
plot_the_loss_curve(epochs, mse)

<class 'dict'>
Epoch 1/20


ValueError: in user code:

    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    H:\Users\Gwydion\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:886 __call__
        self.name)
 